# Extração de dados do Github

Pesquisaremos por iniciativas/projetos que utilizam Dados Abertos Governamentais através da [API do Github](https://developer.github.com/v3/)

A partir das apontamentos feitos na qualificação desse projeto de pesquisa, entendemos que caracterizar toda a comunidade que utiliza dados abertos governamentais é um escopo muito abrangente e de difícil validação.
A ideia é especificar esse escopo para assim poder avaliar melhor os seus resultados, por exemplo verificar se os projetos referências nessas áreas aparecem nos repositórios extraídos no Github.

Por fim, consideramos o contexto de dados abertos governamentais de educação.

O processo de geração de palavras chaves considerou 3 recursos:

- Palavras chaves utilizadas em trabalhos anteriores
- Palavras chaves nas bases de dados do portal do INEP
- Palavras chaves em portais referências para a educação brasileira:
    - [Ministério da Educação](https://www.mec.gov.br/) (sisu, enem, fies, prouni, mec)
    - [Dados Abertos do Ministério da Educação](http://dadosabertos.mec.gov.br/) (pme, prouni, pronatec, pnp, fies)
    - [FUNDEB](https://www.fnde.gov.br/financiamento/fundeb) (fundeb, fnde, siope)
    - [INEP](http://inep.gov.br/dados) (saeb, mde, indicadores financeiros educacionais)
    - [Portal brasileiro de dados abertos](http://www.dados.gov.br/aplicativos): As palavras chaves nesse site já tinham sido cobertas. Porém acho válido tentar contato direto com os projetos listados na seção de Aplicativos.

In [1]:
import requests
import pandas as pd
import time
import logging

As principais palavras chaves usadas por [Attard et al. (2015)](https://www.researchgate.net/publication/281349915_A_Systematic_Review_of_Open_Government_Data_Initiatives) são análise, portal, publicação, consumir juntamente a dados abertos governamentais ou do governo. Porém nos testes com a API do Github *consumir* não se mostrou crucial para retornar resultados relevantes.

In [2]:
first_search_strings = [
            'dados abertos',
            'dados abertos brasil',
            'dados abertos governo',
            'dados abertos governamentais',
            'dados governamentais',
            'dados publicos abertos',
            'dados do governo',
            'analise de dados do governo',
            'analise de dados governamentais',
            'portal de dados do governo',
            'portal de dados governamentais',
            'portal publico do governo',
            'portal de dados abertos do governo',
        ]

In [19]:
actual_search_strings = [
    'dados educacao',
    'dados educacao basica',
    'dados educacionais',
    'analise educacao',
    'analise educacao basica',
    'analise educacional',
    'censo educacao superior',
    'dados educacao superior',
    'analise educacao superior',
    'censo profissionais magistério',
    'dados profissionais magistério',
    'analise profissionais magistério',
    'censo escolar',
    'dados escola inep',
    'dados enade',
    'analise enade',
    'dados encceja',
    'analise encceja',
    'dados enem',
    'analise enem',
    'enem por escola',
    'dados prova brasil',
    'analise prova brasil',    
    'dados ideb',
    'indicadores educacionais',
    'dados ies',
    'analise ies',    
    'dados inep',
    'analise inep',
    'microdados inep',
    'dados sisu',
    'analise sisu',
    'dados fies',
    'analise fies',
    'dados prouni',
    'analise prouni',
    'dados mec',
    'analise mec',
    'dados pme',
    'analise pme',
    'dados pronatec',
    'analise pronatec',
    'dados pnp',
    'analise pnp',
    'dados fundeb',
    'analise fundeb',
    'dados fnde',
    'analise fnde',
    'dados siope',
    'analise siope',
    'dados saeb',
    'analise saeb',
    'dados mde',
    'analise mde',
    'indicadores financeiros educacionais']

In [5]:
len(actual_search_strings)

54

Configuração para gerar arquivo de log

In [6]:
logging.basicConfig(level=logging.DEBUG, 
                    filename="log_file.txt", 
                    filemode="a+",
                    format="%(asctime)s - %(levelname)s - %(funcName)s - %(message)s")

logging.info("Extração de dados do Github")

Para a acesso a alguns recursos da API do github é preciso se autenticar, como aumentar o limite de requisições. Informações sobre autenticação podem ser encontradas [aqui](https://developer.github.com/v3/#authentication).

In [8]:
credentials = ('<user_name>','<token>')

Limite de requisições sem autenticação

In [10]:
t = requests.get('https://api.github.com/rate_limit')
t.json()

{'resources': {'core': {'limit': 60, 'remaining': 59, 'reset': 1588887030},
  'graphql': {'limit': 0, 'remaining': 0, 'reset': 1588888107},
  'integration_manifest': {'limit': 5000,
   'remaining': 5000,
   'reset': 1588888107},
  'search': {'limit': 10, 'remaining': 10, 'reset': 1588884567}},
 'rate': {'limit': 60, 'remaining': 59, 'reset': 1588887030}}

Limite de requisições com autenticação

In [60]:
limits = requests.get('https://api.github.com/rate_limit', auth=credentials)
limits.json()

{'resources': {'core': {'limit': 5000, 'remaining': 5000, 'reset': 1588891296},
  'search': {'limit': 30, 'remaining': 29, 'reset': 1588887696},
  'graphql': {'limit': 5000, 'remaining': 5000, 'reset': 1588891296},
  'integration_manifest': {'limit': 5000,
   'remaining': 5000,
   'reset': 1588891296},
  'source_import': {'limit': 100, 'remaining': 100, 'reset': 1588887756}},
 'rate': {'limit': 5000, 'remaining': 5000, 'reset': 1588891296}}

Verificando limitação de extração de dados da API

In [12]:
page_35 = 'https://api.github.com/search/repositories?q=stars%3A%3E1&sort=stars&order=desc&page=35'
t = requests.get(page_35, auth=credentials)
t.json()

{'message': 'Only the first 1000 search results are available',
 'documentation_url': 'https://developer.github.com/v3/search/'}

Informações sobre a ferramenta de pesquisa da API podem ser encontradas [aqui](https://developer.github.com/v3/search/)

In [13]:
url_base = 'https://api.github.com/search/repositories?q='

Podemos adicionar uma ordenação nos resultados, como quantidade de _stars_ de forma descrescente.

In [14]:
sort = '&sort=stars&order=desc'

## Extraindo informações gerais

In [39]:
def extract_results(data):
    
    items_list = []
    
    logging.info("Debug data keys: {0}".format(data.keys()))
    
    if data.get('message', False):
        logging.info("Debug data message: {0}".format(data.get('message', None)))
        logging.info("Debug data documentation_url: {0}".format(data.get('documentation_url', None)))

    for item in data.get('items', None):
        
        item_dict = {
                'id': item.get('id'),
                'full_name': item.get('full_name', None),
                'description': item.get('description', None),      
                'owner_type': item.get('owner').get('type', None),
                'owner_api_url': item.get('owner').get('url', None),
                'owner_url': item.get('owner').get('html_url', None),
                'api_url': item.get('url', None),
                'url': item.get('html_url', None),
                'fork': item.get('fork', None),
                'created_at': item.get('created_at', None),
                'updated_at': item.get('updated_at', None),
                'pushed_at': item.get('pushed_at', None),
                'size': item.get('size', None),
                'stargazers_count': item.get('stargazers_count', None),
                'language': item.get('language', None),
                'has_issues': item.get('has_issues', None),
                'has_wiki': item.get('has_wiki', None),
                'forks_count': item.get('forks_count', None),
                'forks': item.get('forks', None),
                'open_issues': item.get('open_issues', None),
                'license': item.get('license').get('name', None) if item.get('license', None) else None,
                'timestamp_extract': str(time.time()).split('.')[0]
        }

        items_list.append(item_dict)
            
    return items_list

In [81]:
def check_limit():
    limit = requests.get('https://api.github.com/rate_limit', auth=credentials)
    limit = limit.json().get('resources').get('search').get('remaining')

    if limit == 0: # A API só permite 30 requisições por minuto ao chegar
        time.sleep(180)   

In [82]:
results_by_page = 30

def scroll_pages(url):
    
    check_limit()
    results = requests.get(url, auth=credentials)    
    data = results.json()
    total = data.get('total_count', None)
    
    logging.info("Foram encontrados {0} resultados. Extraindo...".format(total))
        
    items_list = []
    items_list = extract_results(data)
        
    iterations = total // results_by_page 
    
    for iteracao in range(0, iterations):        
        header = results.links
        
        if header.get('next', False):
            next_url = header.get('next').get('url')
            
            check_limit()
            results = requests.get(next_url, auth=credentials)
            data = results.json()
            
            items_list = items_list + extract_results(data)
    
    return items_list

In [ ]:
%%time

items_list = []
repositories_df = None

for index, string in  enumerate(actual_search_strings):
    url = url_base + string + sort
    
    logging.info("Pesquisando repositórios para a string: '{0}'".format(string))
    print('Palavra chave: ', string)

    results = scroll_pages(url)
    items_list = items_list + results
    
    print('Resultados encontrados: ', len(results), '\n')
        
repositories_df = pd.DataFrame(items_list)

In [80]:
repositories_df.tail(3)

,id,full_name,description,owner_type,owner_api_url,owner_url,api_url,url,fork,created_at,...,size,stargazers_count,language,has_issues,has_wiki,forks_count,forks,open_issues,license,timestamp_extract
582,81359776,LeonardoZ/saeb-data,Alternativa ao Software de Análise do Eleitora...,User,https://api.github.com/users/LeonardoZ,https://github.com/LeonardoZ,https://api.github.com/repos/LeonardoZ/saeb-data,https://github.com/LeonardoZ/saeb-data,False,2017-02-08T18:05:16Z,...,13,0,Clojure,True,True,0,0,0,Eclipse Public License 1.0,1588888606
583,66222163,LeonardoZ/saeb,Software de Análise do Eleitorado Brasileiro. ...,User,https://api.github.com/users/LeonardoZ,https://github.com/LeonardoZ,https://api.github.com/repos/LeonardoZ/saeb,https://github.com/LeonardoZ/saeb,False,2016-08-21T21:43:40Z,...,4751,1,Scala,True,True,1,1,0,None,1588888607
585,206887026,datametricks/bartolomeu_saeb,Projeto Professor Bartolomeu - análise dos mic...,User,https://api.github.com/users/datametricks,https://github.com/datametricks,https://api.github.com/repos/datametricks/bart...,https://github.com/datametricks/bartolomeu_saeb,False,2019-09-06T23:12:50Z,...,36922,0,R,True,True,0,0,0,None,1588888607


Quantidade de resultados:

In [76]:
len(repositories_df)

587

Retirando registros duplicados visto que palavras de busca diferentes podem levar a um mesmo repositório.

In [77]:
repositories_df = repositories_df.drop_duplicates(['id', 'api_url'])

In [78]:
len(repositories_df)

401

Quantidade de colunas:

In [74]:
len(repositories_df.columns)

22

In [75]:
repositories_df = repositories_df.sort_values('stargazers_count', ascending=False)

In [76]:
repositories_df.head(1)

,id,full_name,description,owner_type,owner_api_url,owner_url,api_url,url,fork,created_at,...,size,stargazers_count,language,has_issues,has_wiki,forks_count,forks,open_issues,license,timestamp_extract
0,73385196,prefeiturasp/dados-educacao,Análises e tutoriais das bases de dados aberto...,Organization,https://api.github.com/users/prefeiturasp,https://github.com/prefeiturasp,https://api.github.com/repos/prefeiturasp/dado...,https://github.com/prefeiturasp/dados-educacao,False,2016-11-10T13:35:40Z,...,2737,47,Jupyter Notebook,True,True,17,17,2,None,1587003092


## Extraindo _Commits_, _Contributors_ e dados do _Owner_

In [79]:
def extract_commits(url_repo):
    
    commits_url = url_repo + '/commits'  
    results = requests.get(commits_url, auth=credentials)
    
    if results.status_code == 409:
        return None
    
    commits = len(results.json())

    header = results.links
    
    while header.get('next', False):
        next_url = header.get('next').get('url')        
        results = requests.get(next_url, auth=credentials)
        commits = commits + len(results.json())    
        header = results.links


    return commits

In [80]:
def extract_contributors(url_repo):
    
    contributors_url = url_repo + '/contributors'
    results = requests.get(contributors_url, auth=credentials)
    
    if results.status_code == 204:
        return None
    
    contributors = len(results.json())

    header = results.links
    
    while header.get('next', False):
        next_url = header.get('next').get('url')
        results = requests.get(next_url, auth=credentials)
        contributors = contributors + len(results.json())
        header = results.links
    
    return contributors

In [81]:
def extract_owner_data(owner_api_url):
    
    results = requests.get(owner_api_url, auth=credentials)
    data = results.json()

    owner_data = {
        'owner_location': data.get('location', None),
        'owner_email': data.get('email', None),
        'owner_blog': data.get('blog', None),
        'owner_name': data.get('name', None)
    }
    
    return owner_data

In [84]:
%%time
urls = repositories_df['api_url']

for url in urls:

    owner_api_url = repositories_df.loc[repositories_df["api_url"] == url]['owner_api_url'].item()
    owner_data = extract_owner_data(owner_api_url)
    commits = extract_commits(url)
    contributors = extract_contributors(url)
    
    logging.info("Repositório: {0}".format(url))
    logging.info("Tem {0} Commits - {1} Contributors".format(commits,contributors))
    logging.info("Owner location: {0}".format(owner_data.get('owner_location')))

    repositories_df.loc[repositories_df["api_url"] == url, 'commits'] = commits
    repositories_df.loc[repositories_df["api_url"] == url, 'contributors'] = contributors
    repositories_df.loc[repositories_df["api_url"] == url, 'owner_location'] = owner_data.get('owner_location')
    repositories_df.loc[repositories_df["api_url"] == url, 'owner_email'] = owner_data.get('owner_email')
    repositories_df.loc[repositories_df["api_url"] == url, 'owner_blog'] = owner_data.get('owner_blog')
    repositories_df.loc[repositories_df["api_url"] == url, 'owner_name'] = owner_data.get('owner_name')

CPU times: user 36.3 s, sys: 1.26 s, total: 37.6 s
Wall time: 21min 34s


Agora devemos ter mais 6 colunas

In [85]:
len(repositories_df.columns)

28

In [86]:
repositories_df.head(3)

,id,full_name,description,owner_type,owner_api_url,owner_url,api_url,url,fork,created_at,...,forks,open_issues,license,timestamp_extract,commits,contributors,owner_location,owner_email,owner_blog,owner_name
0,73385196,prefeiturasp/dados-educacao,Análises e tutoriais das bases de dados aberto...,Organization,https://api.github.com/users/prefeiturasp,https://github.com/prefeiturasp,https://api.github.com/repos/prefeiturasp/dado...,https://github.com/prefeiturasp/dados-educacao,False,2016-11-10T13:35:40Z,...,17,2,None,1587003092,18.0,2.0,"São Paulo, SP",tecnologia@prefeitura.sp.gov.br,http://www.capital.sp.gov.br,Prefeitura Municipal de São Paulo
409,19895876,inepdadosabertos/api,API de dados abertos para o INEP,Organization,https://api.github.com/users/inepdadosabertos,https://github.com/inepdadosabertos,https://api.github.com/repos/inepdadosabertos/api,https://github.com/inepdadosabertos/api,False,2014-05-17T20:45:05Z,...,4,2,GNU General Public License v2.0,1587003273,29.0,2.0,None,None,,None
2,215114032,Explicador/MozAPI,"API com diversos dados de Moz, incluindo dados...",Organization,https://api.github.com/users/Explicador,https://github.com/Explicador,https://api.github.com/repos/Explicador/MozAPI,https://github.com/Explicador/MozAPI,False,2019-10-14T18:12:48Z,...,0,2,MIT License,1587003092,5.0,1.0,"Maputo, Mozambique",info@explicador.co.mz,https://explicador.co.mz,Explicador


Conferindo valores nulos

In [87]:
len(repositories_df.loc[repositories_df['commits'].isnull()][['api_url', 'commits', 'contributors']])

15

Alguns repositórios realmente não tem nenhum commit como o [Scripts_INEP](https://github.com/ronielsampaio/Scripts_INEP).

In [88]:
repositories_df.loc[repositories_df['contributors'].isnull()][['id', 'url', 'api_url', 'commits', 'contributors']]

,id,url,api_url,commits,contributors
441,168725534,https://github.com/ronielsampaio/Scripts_INEP,https://api.github.com/repos/ronielsampaio/Scr...,NaN,NaN
465,123939819,https://github.com/pessoagabi/inep,https://api.github.com/repos/pessoagabi/inep,NaN,NaN
499,155280861,https://github.com/jhcf/exploradorinep,https://api.github.com/repos/jhcf/exploradorinep,NaN,NaN
369,45876791,https://github.com/dheysonlee/enem2014,https://api.github.com/repos/dheysonlee/enem2014,NaN,NaN
391,124573083,https://github.com/jcosta0/iesf_bd2,https://api.github.com/repos/jcosta0/iesf_bd2,NaN,NaN
398,104570066,https://github.com/amaral08/creative_economy_sp,https://api.github.com/repos/amaral08/creative...,NaN,NaN
83,72317098,https://github.com/grupoccte/Educational-Data-...,https://api.github.com/repos/grupoccte/Educati...,NaN,NaN
123,170711331,https://github.com/igor17400/Data4GoodCPBR12_p...,https://api.github.com/repos/igor17400/Data4Go...,NaN,NaN
26,97905591,https://github.com/mpnetto88/MapeandoEducacao,https://api.github.com/repos/mpnetto88/Mapeand...,NaN,NaN
54,65837427,https://github.com/APSFS/Local-Brazil,https://api.github.com/repos/APSFS/Local-Brazil,NaN,NaN


In [89]:
repositories_df = repositories_df.loc[repositories_df['contributors'].notnull()]
len(repositories_df)

331

Salvando os repositórios

In [90]:
repositories_df.to_csv('../data/repositories_edu.csv', index=False)

## Extraindo contribuidores dos repositórios

In [91]:
def get_contributors(data, repo_data):

    list_contributors = []

    for item in data:        
        contributor = {
            'repo_id': repo_data.get('repo_id', None),
            'repo_name': repo_data.get('repo_name', None),
            'repo_url': repo_data.get('repo_url', None),
            'repo_api_url': repo_data.get('repo_api_url', None),
            'contributor_id': item.get('id', None),
            'contributor_login': item.get('login', None),
            'contributor_type': item.get('type', None),
            'contributor_url': item.get('html_url', None),
            'contributor_api_url': item.get('url', None),
            'timestamp_extract': str(time.time()).split('.')[0]
        }

        list_contributors.append(contributor)

    return list_contributors

In [92]:
def scroll_contributors(url, repo_data):

    list_contributors = []
    results = requests.get(url, auth=credentials)
    
    if results.status_code is 204:
        return None
    
    data = results.json()
    list_contributors = get_contributors(data, repo_data)
    header = results.links
    
    while header.get('next', False):
        
        next_url = header.get('next').get('url')            
        results = requests.get(next_url, auth=credentials)
        data = results.json()
        list_contributors = list_contributors + get_contributors(data, repo_data)  
        header = results.links
        
    return list_contributors

In [93]:
def search_contributors(repositories_df):
    
    urls = repositories_df['api_url']
    list_contributors_all_repo = []
    
    for url in urls:
        logging.info('Extraindo contribuidores de: {0}'.format(url))
        
        repo_data = {
                'repo_id': repositories_df.loc[repositories_df["api_url"] == url, 'id'].values[0],
                'repo_name': repositories_df.loc[repositories_df["api_url"] == url, 'full_name'].values[0],
                'repo_url': repositories_df.loc[repositories_df["api_url"] == url, 'url'].values[0],
                'repo_api_url': url,
            }
        
        url_contributors = url + '/contributors'        
        contributors = scroll_contributors(url_contributors, repo_data)
        
        if contributors:
            list_contributors_all_repo = list_contributors_all_repo + contributors
    
    contributors_df = pd.DataFrame(list_contributors_all_repo)     
        
    return contributors_df

In [94]:
%%time
contributors_df = search_contributors(repositories_df)

CPU times: user 9.75 s, sys: 320 ms, total: 10.1 s
Wall time: 8min 52s


In [95]:
contributors_df.head(3)

,repo_id,repo_name,repo_url,repo_api_url,contributor_id,contributor_login,contributor_type,contributor_url,contributor_api_url,timestamp_extract
0,73385196,prefeiturasp/dados-educacao,https://github.com/prefeiturasp/dados-educacao,https://api.github.com/repos/prefeiturasp/dado...,2335525,campagnucci,User,https://github.com/campagnucci,https://api.github.com/users/campagnucci,1587005987
1,73385196,prefeiturasp/dados-educacao,https://github.com/prefeiturasp/dados-educacao,https://api.github.com/repos/prefeiturasp/dado...,28656406,tutss,User,https://github.com/tutss,https://api.github.com/users/tutss,1587005987
2,19895876,inepdadosabertos/api,https://github.com/inepdadosabertos/api,https://api.github.com/repos/inepdadosabertos/api,7613695,NandaBeirao,User,https://github.com/NandaBeirao,https://api.github.com/users/NandaBeirao,1587005988


Verificando se há contribuidores repetidos para um mesmo repositório.

In [96]:
contributors_df[contributors_df.duplicated(['contributor_id', 'repo_id'])]

,repo_id,repo_name,repo_url,repo_api_url,contributor_id,contributor_login,contributor_type,contributor_url,contributor_api_url,timestamp_extract


In [98]:
len(contributors_df)

436

Salvando dataframe com mapeamento de repositórios e contribuidores.

In [97]:
contributors_df.to_csv('../data/contributors_edu.csv', index=False)